In [18]:
import openai
import time
import random
import numpy as np
import pandas as pd
from common import load_api_key
from pathlib import Path


In [2]:
api_key = load_api_key()
openai.api_key = api_key

In [3]:
news_df = pd.read_csv(r"C:\Users\Rui\Documents\Explainable_AI\ExplainableNRS\dataset\MIND\small\news.csv")

In [6]:
sampled_df = pd.read_csv("sampled_100.csv")

In [25]:
def generate_correct_random_predictions(candidate_col):
    """Generate random recommendations based on the candidate column with correct format."""
    return candidate_col.apply(
        lambda x: ','.join([f"C{item + 1}" for item in random.sample(range(len(x.split("\n"))), len(x.split("\n")))])
    )


,Metric,Random Predictions,GPT-3.5,GPT-4
0,nDCG@5,0.222190,0.210149,0.413043
1,nDCG@10,0.302553,0.284718,0.455982
2,MRR,0.259391,0.196758,0.382063


In [ ]:
#!/bin/bash -l
model_name=models/chat-bison-001
sample_num=100
max_num=1
prompt_temp=naive_zero_shot
python request_api.py --model_name=$model_name --sample_num=$sample_num --max_num=$max_num --prompt_temp=$prompt_temp